In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Imports

In [0]:
import collections
import pickle
import gensim
from gensim.models.keyedvectors import KeyedVectors
import numpy as np
import re
import random
import numpy as np
import sys
import os 

# Implementation

In [0]:
cd /content/drive/My Drive/University/FYP/Sentiment Analysis/supportive/S-LSTM/classfication

/content/drive/My Drive/University/FYP/Sentiment Analysis/supportive/S-LSTM/classfication


In [3]:
def make_dir(path):
    if not os.path.exists(path):
        os.mkdir(path)
        print('made directory %s' % path)
make_dir("parsed_data")

made directory parsed_data


In [0]:
def text_preprocessing(datasets):
	dataset_text=[]
	dataset_label=[]
	for file in datasets:
		lines=[]
		labels=[]
		with open(file) as f:
			for l in f:
				try:
					words=re.split('\s|-',l.lower().split("|||")[0].strip())
					words=words
					label=int(l.lower().split("|||")[1].strip())
					lines+=[words]
					labels+=[label]
				except:
					continue
		dataset_text+=[lines]
		dataset_label+=[labels]
	return dataset_text, dataset_label

In [0]:
def insert_word(dataset, all_words):
	for lines in dataset:
		for l in lines:
			all_words+=l

In [0]:
def convert_words_to_number(dataset_text, dataset_label, common_word):
	transformed_text=[]
	transformed_label=[]
	for lines, labels in zip(dataset_text, dataset_label):
		new_x=[]
		new_label=[]
		for l, label in zip(lines,labels):
			words=[common_word[w] if w in common_word else 1 for w in l]	
			
			new_x+=[words]
			new_label+=[label]

		transformed_text+=[new_x]
		transformed_label+=[new_label]
	return transformed_text, transformed_label

# Main

In [0]:
datasets=['/content/drive/My Drive/University/FYP/Sentiment Analysis/supportive/S-LSTM/classfication/sst_data/apparel_trn',
          '/content/drive/My Drive/University/FYP/Sentiment Analysis/supportive/S-LSTM/classfication/sst_data/apparel_dev',
          '/content/drive/My Drive/University/FYP/Sentiment Analysis/supportive/S-LSTM/classfication/sst_data/apparel_tst'
          ]

In [0]:
#preprocess the texts
dataset_text, dataset_label=text_preprocessing(datasets)

In [10]:
print(dataset_label)

[[0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1,

In [0]:
#insert all words
all_words=[]
insert_word(dataset_text, all_words)

In [0]:
#obtain frequent words
counter=collections.Counter(all_words)
vocab=len(counter)
vocab_size=vocab-2	
common_word=dict(counter.most_common(vocab_size))
print(len(common_word))

8101


In [0]:
#number them
c=2
for key in common_word:
  common_word[key]=c
  c+=1

In [0]:
argument1 = 'apparel'

#write out filtering training test data
transformed_text, transformed_label= convert_words_to_number(dataset_text, dataset_label, common_word)

pickle.dump(((transformed_text, transformed_label)), open('/content/drive/My Drive/University/FYP/Sentiment Analysis/supportive/S-LSTM/classfication/parsed_data'+argument1+'_dataset', 'wb'))

glove_filename="/content/drive/My Drive/University/FYP/Sentiment Analysis/supportive/S-LSTM/Word_Vectors/glove.6B.300d.txt"
glove_dict={}
with open(glove_filename) as f:
  for line in f:
    line = line.strip().split(' ')
    word = line[0]
    embedding = [float(x) for x in line[1:]]
    glove_dict[word]=embedding

word2vec=[np.random.normal(0, 0.1, 300).tolist(), np.random.normal(0, 0.1, 300).tolist()]
missing=0
for number, word in sorted(zip(common_word.values(), common_word.keys())):
  try:
    word2vec.append(glove_dict[word])
  except KeyError: 	
    word2vec.append(np.random.normal(0, 0.1, 300).tolist())
    missing+=1
	#print(number)
  print(len(word2vec))
pickle.dump(word2vec, open('/content/drive/My Drive/University/FYP/Sentiment Analysis/supportive/S-LSTM/classfication/parsed_data'+argument1+'_vectors', 'wb'))
print(missing)
print(np.array(word2vec).shape)

# Downloading Glove Vectors

In [0]:
cd /content/drive/My Drive/University/FYP/Sentiment Analysis/supportive/S-LSTM/Word_Vectors

/content/drive/My Drive/University/FYP/Sentiment Analysis/supportive/S-LSTM/Word_Vectors


In [0]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-02-22 16:57:10--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-02-22 16:57:10--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-02-22 16:57:11--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [0]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       
